<a href="https://colab.research.google.com/github/Sakinat-Folorunso/OOU_DTS201_Introduction_to_Data_Science/blob/main/notebooks/DTS201_Week5_PH_Student_Centred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DTS 201 – INTRODUCTION TO DATA SCIENCE  
## WEEK 5 PRACTICAL (PH): PREDICTION AND SIMPLE REGRESSION MODELLING

**Instructor:** DR SAKINAT FOLORRUNSO – ASSOCIATE PROFESSOR OF AI SYSTEMS AND FAIR DATA  
**Department:** COMPUTER SCIENCES, OLABISI ONABANJO UNIVERSITY, AGO-IWOYE, OGUN STATE, NIGERIA  

---

### 🎯 Practical Learning Objectives (Week 5)

By the end of this practical session, you should be able to:

- Explain the basic idea of **prediction** and how it differs from description.  
- Identify a **target variable** and **predictor feature(s)** in a dataset.  
- Split data into **training** and **test** sets.  
- Build a simple **linear regression** model using Python.  
- Evaluate model performance using **Mean Squared Error (MSE)**, **Mean Absolute Error (MAE)**, and **R² score**.  
- Visualize a regression line on a scatter plot and interpret the relationship.

> 💡 This notebook uses Python’s `scikit-learn` to mirror the “simple regression model in R” described in the course outline.


## 1. Getting Started: Choosing a Dataset for Prediction

In this practical, you will need a dataset that contains:

- At least **one numerical feature** (predictor), e.g. `Hours_Studied`, `Age`, `Years_of_Experience`, `Square_Footage`.  
- One **numerical target variable** (what you want to predict), e.g. `Exam_Score`, `Salary`, `House_Price`, `Sales_Amount`.

You may use:

- A **students performance dataset** (e.g. Hours studied → Exam score).  
- A simple **sales dataset** (e.g. Advertising spend → Sales).  
- Any other pair of variables where a linear relationship is reasonable.

> ✅ **Goal:** Build a simple regression model with **one predictor** and **one target** to understand the full pipeline from data to prediction.


### 1.1 Optional: Mount Google Drive (If Using Google Colab)

In [ ]:
# Run this cell ONLY if:
# 1. You are working in Google Colab, AND
# 2. Your dataset is stored in your Google Drive.

# Import the drive module from google.colab for mounting Google Drive
from google.colab import drive  # Provides tools to connect Colab to your Google Drive

# Mount your Google Drive so that its files are accessible inside Colab
drive.mount('/content/drive')  # Follow the instructions and paste the authorization code when prompted

# After mounting, your files can be accessed under '/content/drive/MyDrive/'.
# Example path: data_path = '/content/drive/MyDrive/Datasets/students_scores.csv'

## 2. Importing Required Libraries

In [ ]:
# Import pandas for loading and manipulating tabular data
import pandas as pd

# Import numpy for numerical computations
import numpy as np

# Import matplotlib for plotting graphs
import matplotlib.pyplot as plt

# Import seaborn for more advanced and attractive statistical visualizations
import seaborn as sns

# Import tools from scikit-learn for machine learning tasks
from sklearn.model_selection import train_test_split  # For splitting data into training and test sets
from sklearn.linear_model import LinearRegression      # For building a simple linear regression model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score  # For evaluating model performance

# Ensure that plots appear inside the notebook
%matplotlib inline

# Set a visual style for plots to improve readability
plt.style.use("seaborn-v0_8")

## 3. Loading the Dataset

> 🔧 **Instruction:**  
> - Replace the file name in `data_path` with the actual name of your dataset (e.g. `students_scores.csv`).  
> - Make sure the file is in the same folder as this notebook, or provide the full path.  
> - Your dataset should contain at least one suitable predictor and one target variable.


In [ ]:
# Specify the path or file name of your dataset
# Example (local): data_path = "students_scores.csv"
# Example (Colab + Drive): data_path = "/content/drive/MyDrive/Datasets/students_scores.csv"
data_path = "students_scores.csv"  # TODO: change this to the correct path or file name

# Load the dataset from CSV into a pandas DataFrame
df = pd.read_csv(data_path)  # Reads the CSV file and stores its contents in 'df'

# Display the first 5 rows of the dataset to inspect its structure
df.head()  # Helps you identify column names and get a feel for the data

> ✏️ **Student Task 1:**  
> - What does **one row** of this dataset represent?  
> - List at least **three columns** and describe what each one contains.  
> - Identify which column you intend to predict (the **target**) and which column you will use as the **predictor**.


### 3.1 Checking the Structure of the Dataset

In [ ]:
# Check how many rows and columns the dataset has
df.shape  # Returns a tuple: (number_of_rows, number_of_columns)

In [ ]:
# Display summary information about the dataset
# Includes column names, data types, and non-null counts
df.info()  # Helps you identify which columns are numerical or categorical

In [ ]:
# View basic summary statistics for numerical columns
df.describe()  # Shows count, mean, std, min, max, and quartiles for numeric features

## 4. Choosing Predictor and Target Variables

For simple linear regression, we will:

- Use **one numerical column** as the **predictor** (input feature `X`).  
- Use **one numerical column** as the **target** (output variable `y`).  

Examples:

- Predictor: `Hours_Studied`, Target: `Exam_Score`  
- Predictor: `Years_of_Experience`, Target: `Salary`  
- Predictor: `Advertising_Spend`, Target: `Sales`  


In [ ]:
# Set the names of your predictor (feature) and target columns
# Replace 'Hours_Studied' and 'Exam_Score' with real column names from your dataset
feature_col = "Hours_Studied"  # TODO: change this to your chosen predictor column
target_col = "Exam_Score"      # TODO: change this to your chosen target column

# Create the feature matrix X (must be 2D for scikit-learn)
X = df[[feature_col]]  # Double brackets keep X as a DataFrame with one column

# Create the target vector y (can be 1D)
y = df[target_col]     # Single bracket creates a Series representing the target

In [ ]:
# Display the first few rows of X and y to confirm they look correct
print("Feature (X) sample:")
print(X.head())  # Shows the first 5 values of the predictor

print("
Target (y) sample:")
print(y.head())  # Shows the first 5 values of the target

> ✏️ **Student Task 2:**  
> - Write down in words what `feature_col` and `target_col` represent in your chosen dataset.  
> - Do you expect a **positive**, **negative**, or **no clear** relationship between them? Why?


## 5. Splitting Data into Training and Test Sets

To evaluate how well our model **generalizes** to unseen data, we split the dataset into:

- **Training set** – used to fit (train) the model.  
- **Test set** – used to evaluate model performance on data it has not seen before.

A common split is **80% training** and **20% test**.


In [ ]:
# Split the data into training and test sets
# test_size=0.2 means 20% of the data will be used for testing, 80% for training
# random_state is set so that the split is reproducible
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Display the sizes of each split to check how many samples are in train and test sets
print("Training set size:", X_train.shape[0], "samples")
print("Test set size:", X_test.shape[0], "samples")

## 6. Building and Training a Simple Linear Regression Model

We will fit a **Linear Regression** model of the form:

\[ \hat{y} = b_0 + b_1 x \]

where:

- \( x \) is the predictor (feature).  
- \( \hat{y} \) is the predicted target value.  
- \( b_0 \) is the intercept (prediction when x = 0).  
- \( b_1 \) is the slope (how much \( y \) changes when \( x \) increases by 1 unit).


In [ ]:
# Create an instance of the LinearRegression model
lin_reg = LinearRegression()  # Initializes a new linear regression model

# Fit (train) the model using the training data
lin_reg.fit(X_train, y_train)  # The model learns the best-fitting line for X_train → y_train

# Retrieve the learned intercept (b0) and slope (b1)
intercept = lin_reg.intercept_    # This is b0 in the equation y = b0 + b1*x
slope = lin_reg.coef_[0]          # This is b1, the slope for our single feature

# Print the model parameters
print("Learned model:")
print(f"Intercept (b0): {intercept:.3f}")
print(f"Slope (b1): {slope:.3f}")

print(f"
Regression equation: {target_col} ≈ {intercept:.3f} + {slope:.3f} × {feature_col}")

> ✏️ **Student Task 3:**  
> - Interpret the slope in plain language. For example: “For every one-unit increase in `feature_col`, the model predicts that `target_col` increases by approximately … units.”  
> - Does this interpretation make **real-world sense** in your context?


## 7. Making Predictions on the Test Set

In [ ]:
# Use the trained model to make predictions on the test features
y_pred = lin_reg.predict(X_test)  # Produces predicted values corresponding to X_test

# Display the first few predicted values alongside the true values
results_sample = pd.DataFrame({
    "Actual": y_test.values[:10],      # First 10 true target values
    "Predicted": y_pred[:10]          # First 10 predicted target values
})

results_sample  # Shows a small comparison table of actual vs predicted

## 8. Evaluating Model Performance

We will use three common regression evaluation metrics:

- **Mean Squared Error (MSE):** Average of squared errors. Penalizes large errors more.  
- **Mean Absolute Error (MAE):** Average of absolute errors. Easier to interpret in original units.  
- **R² Score:** Proportion of variance in the target explained by the model (ranges from 0 to 1 for many cases).  


In [ ]:
# Compute Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)  # Average of squared differences between actual and predicted

# Compute Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)  # Average of absolute differences between actual and predicted

# Compute R-squared (R²) score
r2 = r2_score(y_test, y_pred)  # Measures how much of the variance in y is explained by the model

# Print the evaluation metrics
print("Model Evaluation Metrics:")
print(f"Mean Squared Error (MSE): {mse:.3f}")
print(f"Mean Absolute Error (MAE): {mae:.3f}")
print(f"R-squared (R²): {r2:.3f}")

> ✏️ **Student Reflection 4:**  
> - Is the MAE small or large relative to the typical values of your target variable?  
> - Is your R² closer to 0 (poor fit) or 1 (good fit)?  
> - Based on these metrics, how well is your model performing?


## 9. Visualizing the Regression Line

A useful way to understand the model is to plot:

- The **actual data points** (scatter plot).  
- The **regression line** predicted by the model.

We will do this using the training data.


In [ ]:
# Create a scatter plot of the training data
plt.figure(figsize=(7, 5))  # Set the figure size

# Plot the actual data points as a scatter plot
plt.scatter(X_train, y_train, label="Training data")  # Each dot is one observation

# Create a sequence of x-values spanning the range of the feature
x_line = np.linspace(X[feature_col].min(), X[feature_col].max(), 100).reshape(-1, 1)

# Use the model to predict y-values for this sequence of x-values
y_line = lin_reg.predict(x_line)

# Plot the regression line
plt.plot(x_line, y_line, color="red", label="Regression line")  # Line showing the model's predictions

# Add title and labels
plt.title(f"Regression Line: {target_col} vs {feature_col}")
plt.xlabel(feature_col)
plt.ylabel(target_col)

# Show the legend to distinguish data points from regression line
plt.legend()

# Display the plot
plt.tight_layout()
plt.show()

> ✏️ **Student Task 5:**  
> - Look at the scatter plot and regression line. Do most points lie close to the line or far away?  
> - Are there any **outliers** that are far from the line? How might they affect the model?


## 10. Extension: Trying a Different Predictor (Optional)

Sometimes, another predictor may work better for predicting the target.

> 🧪 **Suggested Activity:**  
> - Choose a different numerical column as `feature_col`.  
> - Re-run Sections 4–9 (you only need to change the variable once).  
> - Compare the new model’s MSE, MAE, and R² with the original model.  
> - Decide which predictor is more useful for your target.


## 11. Mini Regression Exercise (Apply the Full Pipeline)

> 🧪 **Student Activity:**  
> Using your chosen dataset, complete the following steps and prepare a brief report (bullet points or paragraph):

1. Clearly state your **prediction problem** (e.g., “Predict exam score from hours studied”).  
2. Identify the **predictor** and **target** columns.  
3. Split the data into training and test sets.  
4. Train a linear regression model.  
5. Report MSE, MAE, and R².  
6. Include a scatter plot with the regression line.  
7. Write **3–5 sentences** interpreting the results and limitations of your model.


## 12. Summary and Reflection (Week 5 PH)

In this Week 5 practical, you have:

- Identified predictor and target variables in a dataset.  
- Split data into training and test sets to evaluate generalization.  
- Trained a simple linear regression model.  
- Evaluated performance using MSE, MAE, and R².  
- Visualized the regression line and interpreted the relationship between variables.

> 🧠 **Final Reflection (write 1–2 paragraphs):**  
> - In your own words, what is the goal of a prediction model?  
> - How would you explain the meaning of “overfitting” to a non-technical friend (even if we did not fully explore it here)?  
> - How do you think more complex models (with more features or non-linear relationships) might improve or complicate prediction?  
